In [1]:
!conda install simpletransformers

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - simpletransformers

Current channels:

  - https://repo.anaconda.com/pkgs/main/osx-arm64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/osx-arm64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [1]:
import os
import pandas as pd

import transformers
import torch
import numpy as np
from simpletransformers.language_representation import RepresentationModel
from tqdm import tqdm

import umap.umap_ as umap
import matplotlib.pyplot as plt
import sklearn.cluster as cluster
import json
import string

In [2]:
os.chdir('..')

In [3]:
os.listdir('./')

['thing.txt',
 'searcher',
 'bash_docker_start_app.sh',
 '.DS_Store',
 'requirements_macos.txt',
 'requirements.txt',
 'c-nov.json',
 'test_save_rslts_100.json',
 'learningmachines',
 'lda_hyperparam_eval.py',
 'etc',
 'db.sqlite3',
 'static',
 'bash_docker_start_worker.sh',
 'bert_scripts',
 'running_tester.py',
 'manage.py',
 '.ipynb_checkpoints']

In [4]:
from searcher.es_search import SearchResults_ES

{'USE_S3': False, 'BUCKET_NAME': 'learningmachines-static', 'CUSTOM_DOMAIN': 'learningmachines-static.s3.amazonaws.com'}
credentials file found
redis://localhost:6379
https://learningmachines-static.s3.amazonaws.com/static/
learning-machines-


/Users/ez/Desktop/projects/learningmachines_folder/venv/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [5]:
#https://simpletransformers.ai/docs/lm-model/

#https://www.sbert.net/examples/applications/computing-embeddings/README.html

#https://umap-learn.readthedocs.io/en/latest/clustering.html

In [6]:
import json
from collections import namedtuple
class EsLocalRslt:
    def __init__(self, save_qry_obj, save_qry_path=None):
        print(save_qry_obj)
        self.save_qry_path = f"bert_scripts/data/{save_qry_obj['qry']}_{save_qry_obj['database']}_{save_qry_obj['maximum_hits']}_{save_qry_obj['qry']}.json" if save_qry_path == None else save_qry_path
        self.save_qry_obj = save_qry_obj
        self.es = SearchResults_ES(save_qry_obj['database'], save_qry_obj, cleaned=False, rand=False)
        self.rslts = []
        self.esDocRslt = []
        self.esDoc = namedtuple('esDoc', ['doc_id', 'journal_title', 'article_title', 'authors', 'date', 'text', 'doi'])

    def save_rslts(self):
        self.rslts = []
        for d in self.es:
            self.rslts.append(d)
    
        fp = open(self.save_qry_path, 'w')
        json.dump(self.rslts, fp)
        
    def load_rslts(self):
        rfp = open(self.save_qry_path, 'r')
        self.rslts = json.load(rfp)
        self.esDocRslt = []
        for d in self.rslts:
            esD = self.esDoc(
                    doc_id=d[0],
                    journal_title=d[1],
                    article_title=d[2],
                    authors=d[3],
                    text=d[5],
                    date=d[4],
                    doi=d[6]
                )
            self.esDocRslt.append(esD)
        return self.esDocRslt




In [7]:
test_qry_obj = {'qry': '', 'maximum_hits': '10000', 'database': 'chicago-novels'}


dd = EsLocalRslt(test_qry_obj)
dd.save_rslts()

{'qry': '', 'maximum_hits': '10000', 'database': 'chicago-novels'}
10000
{'qry': '', 'maximum_hits': '10000', 'database': 'chicago-novels', 'f_start': -1, 'f_end': -1, 'min_occurrence': -1, 'max_occurrence': -1}
{'match_all': {}}
search query {'size': 250, 'query': {'match_all': {}}}
10000
{'qry': '', 'maximum_hits': '10000', 'database': 'chicago-novels', 'f_start': -1, 'f_end': -1, 'min_occurrence': -1, 'max_occurrence': -1}
{'match_all': {}}
search query {'size': 250, 'query': {'match_all': {}}}
scroll 0 250
10000
{'qry': '', 'maximum_hits': '10000', 'database': 'chicago-novels', 'f_start': -1, 'f_end': -1, 'min_occurrence': -1, 'max_occurrence': -1}
{'match_all': {}}
search query {'size': 250, 'query': {'match_all': {}}}
scroll 1 250
10000
{'qry': '', 'maximum_hits': '10000', 'database': 'chicago-novels', 'f_start': -1, 'f_end': -1, 'min_occurrence': -1, 'max_occurrence': -1}
{'match_all': {}}
search query {'size': 250, 'query': {'match_all': {}}}
scroll 2 250
10000
{'qry': '', 'max

scroll 34 250
10000
{'qry': '', 'maximum_hits': '10000', 'database': 'chicago-novels', 'f_start': -1, 'f_end': -1, 'min_occurrence': -1, 'max_occurrence': -1}
{'match_all': {}}
search query {'size': 250, 'query': {'match_all': {}}}
scroll 35 89


In [9]:
local_rslts = dd.load_rslts()
print(len(local_rslts))

9089


In [10]:
 class EmbGenerator:
    def __init__(self, model_name, qry_obj, max_length=512, proj_method='umap', cluster_method='kmeans', local_rslts=None):
        self.model = RepresentationModel(model_type="bert", model_name=model_name, use_cuda=False)
        self.max_length = max_length
        self.qry_obj = qry_obj
        self.es = SearchResults_ES(qry_obj['database'], qry_obj, cleaned=False, rand=False) if local_rslts == None else local_rslts
        self.proj_method = proj_method
        self.cluster_method = cluster_method
        self.doc_embs = {'x' : [],
                         'y' : [],
                         'emb' : [],
                         'title' : [],
                         'dates' : [],
                         'id' : [],
                        'clusters' : [],
                        'fulltexts': [],
                        'cluster_labels_docs' : [],
                        'cluster_labels_tfidf' : [],
                        'cluster_centroids' : []}
        self.num_clusters=10
        self.cluster_docs_idxs = {}
        self.dim_fit = None
        
        self.rmchars = string.punctuation + "º"
        self.rmchars = self.rmchars.replace("_", "").replace("-", "").replace("%", "")
        
    def clean_text(self, doc):
        doc_text = doc.lower()
        if doc_text == None:
            doc_text = "" 
        split_text = doc_text.lower().translate(str.maketrans("", "", self.rmchars))
        split_text = split_text.replace("-", " ").split(" ") if doc_text != None else []

        if len(split_text) > 5000:
            split_text = split_text[0:5000]
        cleaned_text = split_text
        return cleaned_text
    
    def create_doc_obj(self, emb, doc_info):
        self.doc_embs['emb'].append(emb)
        self.doc_embs['title'].append(doc_info.article_title.replace('"', "").replace("\n", ""))
        self.doc_embs['id'].append(doc_info.doc_id)
        self.doc_embs['dates'].append(doc_info.date)
        if type(self.es) == list:
            self.doc_embs['fulltexts'].append(self.clean_text(doc_info.text))
        return
    
    def split_sentences(self, doc):
        split_doc = doc.split()
        chunked_list = list()
        chunk_size = self.max_length
        for i in range(0, len(split_doc), chunk_size):
            chunked_list.append(" ".join(split_doc[i:i+chunk_size]))

        return(chunked_list[:25] if len(chunked_list) > 15 else chunked_list)
    
    def generate_embs(self):
        for document in tqdm(self.es):
            dsplits = self.split_sentences(document.text)
            sentenceVectors = self.model.encode_sentences(dsplits, combine_strategy="mean")
            word_embedding_avg = np.mean(sentenceVectors, axis=0)
            self.create_doc_obj(word_embedding_avg, document)
            
    def reduce_dims(self, plot=False):
        if self.proj_method == 'umap':
            import umap.umap_ as umap
            self.dim_fit = umap.UMAP(
                n_neighbors=15,
                min_dist=.1,
                n_components=2,
                metric='cosine')
            
            self.dim_fit.fit(self.doc_embs['emb'])
            u = self.dim_fit.transform(self.doc_embs['emb'])
            self.doc_embs['x'] = u[:,0].tolist()
            self.doc_embs['y'] = u[:,1].tolist()
        
    def cluster_embs(self, reduced=False):
        if reduced:
            X = self.doc_embs['x']
        else:
            X = self.doc_embs['emb']
        
        if self.cluster_method == 'kmeans':
            self.doc_embs['clusters'] = cluster.KMeans(n_clusters=self.num_clusters, random_state=0).fit_predict(X).tolist()
        if type(self.cluster_method) == list:
            self.doc_embs['clusters'] == self.cluster_method
            
    def _doc_centroid_labels(self, nearest_doc_label=True):
        from sklearn.neighbors import NearestCentroid
        from sklearn.neighbors import NearestNeighbors
        X = np.array(eg.doc_embs['emb'])
        y = np.array(eg.doc_embs['clusters'])
        clf = NearestCentroid()
        clf.fit(X, y)
        self.doc_embs['cluster_centroids'] = self.dim_fit.transform(clf.centroids_).tolist()
        
        if nearest_doc_label == True:
            combined_embs = np.concatenate((clf.centroids_, X), axis=0)
            knn = NearestNeighbors(n_neighbors=self.num_clusters + 1)
            knn.fit(combined_embs)
            
            for i in range(self.num_clusters):
                neighbors = knn.kneighbors([combined_embs[i]], return_distance=False)
                added = False
                for n in neighbors[0]:
                    if n > self.num_clusters:
                        self.doc_embs['cluster_labels_docs'].append(self.doc_embs['title'][n - self.num_clusters])
                        added = True
                        break
                if added == False:
                    print(neighbors)
                    self.doc_embs['cluster_labels_docs'].append('error in clustering')
        return
    
    #https://nlp.stanford.edu/IR-book/html/htmledition/cluster-labeling-1.html
    #TODO implement internal measures like above
    
    def _tfidf_cluster_labels(self):
        #unimplemented
        from gensim.models import TfidfModel
        from gensim.corpora import Dictionary
        dct = Dictionary(self.doc_embs["fulltexts"]) 
        cluster_docs = []

        for c in self.cluster_docs_idxs.keys():
            cluster_doc = []
            for d in eg.cluster_docs_idxs[c]:
                cluster_doc = cluster_doc + self.doc_embs["fulltexts"][d]
            cluster_docs.append(cluster_doc)
        corpus = [dct.doc2bow(c_d) for c_d in cluster_docs ]
        model = TfidfModel(corpus)

        for c_d in corpus:
            tfidf_c = model[c_d]

            tfidf_c = sorted(tfidf_c, key=lambda x : x[1], reverse=True)
            joined_c_tfidf = " ".join(list(map(lambda x: dct[x[0]], tfidf_c[0:5])))
            self.doc_embs['cluster_labels_tfidf'].append(joined_c_tfidf)
        return
    
    def label_clusters(self, method="relative"):
        if len(self.doc_embs['clusters']) == 0:
            return "You need to cluster your embeddings first!, run cluster_embs"
        unique_clusters = list(set(self.doc_embs['clusters']))
        
        self.cluster_docs_idxs = {}
        for c in unique_clusters:
            indices  = [index for (index, item) in enumerate(self.doc_embs['clusters']) if item == c]
            self.cluster_docs_idxs[c] = indices   
        
        self._doc_centroid_labels()
        self._tfidf_cluster_labels()
            
    def dump_embs(self):
        cc = self.cluster_method if type(self.cluster_method) == str else 'custom'
        f = open(f"searcher/static/searcher/bert_method_data/{self.qry_obj['qry']}_{self.qry_obj['database']}_{self.proj_method}_{cc}.json", 'w')
        self.doc_embs['emb'] = []
        self.doc_embs['fulltexts'] = []
        json.dump(self.doc_embs, f)
        f.close()
        

model_name = 'bert-base-uncased'
eg = EmbGenerator(model_name, test_qry_obj, max_length=512, local_rslts=local_rslts)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTextRepresentation: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
eg.generate_embs()

  9%|███▍                                  | 821/9089 [14:53<2:30:03,  1.09s/it]

In [ ]:
eg.reduce_dims()

In [ ]:
eg.cluster_embs()

In [ ]:
eg.label_clusters()

In [ ]:
eg.dump_embs()

In [ ]:
len(eg.doc_embs['x'])

In [ ]:
eg.doc_embs['cluster_labels_docs']

In [ ]:
eg.doc_embs['cluster_labels_tfidf']
    

In [54]:
for x in range(eg.num_clusters):
    print(x)
    print(eg.doc_embs['cluster_labels_docs'][x])
    print(eg.doc_embs['cluster_labels_tfidf'][x])

0
The English nvnne being a treatise wherein (by way of dialogue) the author endeauoureth to draw yong & vnmarried Catholike gentlewomen to imbrace a votary and religious life / written by N.N. Hereunto is annexed a short discourse (by way of conclusion) to the abbesses and religious women of all the English monasteries in the Low-countreys and France.
of and to in that
1
The transactioneer, with some of his philosophical fancies in two dialogues.
and of to a in
2
Julian's arts to undermine and extirpate Christianity together with answers to Constantius the Apostate, and Jovian / by Samuel Johnson.
of and to in that
3
A notable sermo[n] of ye reuerende father Maister Hughe Latemer whiche he preached in ye Shrouds at paules churche in Londo[n], on the. xviii. daye of Ianuary. 1548.
of and to whiche hym
4
The reign of King Charles an history faithfully and impartially delivered and disposed into annals.
and of to in that
5
Diatriba chronologica Johannis Marshami
vous amp est ✚ non
6
The 

In [38]:
test_doc = eg.doc_embs['emb'][0]

In [39]:
test_doc.shape

(768,)

In [40]:
type(test_doc)

numpy.ndarray

In [57]:
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import NearestNeighbors

In [84]:
X = np.array(eg.doc_embs['emb'])
y = np.array(eg.doc_embs['clusters'])
clf = NearestCentroid()
clf.fit(X, y)
 
`
clf.centroids_.shape

(10, 768)

In [85]:
type(clf.centroids_)

numpy.ndarray

In [86]:
type(X)

numpy.ndarray

In [89]:
combined_embs = np.concatenate((clf.centroids_, X), axis=0)

In [90]:
combined_embs.shape

(110, 768)

In [114]:
knn = NearestNeighbors(n_neighbors=5)

In [115]:
knn.fit(combined_embs)

NearestNeighbors()

In [121]:
rslt = knn.kneighbors([combined_embs[9]], return_distance=False)

rslt[0][1] - 10


29

In [ ]:
XX = np.array[]

In [56]:
import numpy as np
from numpy.linalg import norm

 
# define two lists or array
A = np.array(eg.doc_embs['emb'][5])
B = np.array(clf.centroids_[y[0]])
 
print("A:", A)
print("B:", B)
 
# compute cosine similarity
def get_cosine(A,B)
    cosine = np.dot(A,B)/(norm(A)*norm(B))
    return cosine


print("Cosine Similarity:", cosine)

A: [-1.43004119e-01  1.14563979e-01  1.45867229e-01 -2.04900369e-01
  3.44652608e-02 -1.19383693e-01  2.45175600e-01  8.42082426e-02
 -1.22662850e-01  1.80233084e-03 -1.57775447e-01 -4.11728531e-01
 -1.87742829e-01  4.24652994e-02  8.72433782e-02  4.32404608e-01
  4.22497958e-01 -1.68222994e-01 -1.38846368e-01  3.29564512e-01
  2.43533492e-01 -1.59442008e-01  2.04009145e-01  6.48970962e-01
  4.34518963e-01  9.36256200e-02  4.67887104e-01  1.21314473e-01
 -5.17959222e-02 -8.58383067e-03  7.43761957e-01  2.22229511e-01
  1.72250032e-01 -8.04997757e-02  1.18688226e-01  1.29513144e-01
 -1.45678613e-02 -1.64983019e-01  9.27333608e-02  2.68303812e-01
 -2.94840068e-01 -5.26194572e-01 -2.77666867e-01 -1.49034664e-01
 -2.13052303e-01 -4.41331178e-01  2.82821685e-01  2.17839442e-02
  1.67615235e-01  1.22373782e-01 -3.21592122e-01  4.31020290e-01
  1.89112201e-02 -1.85902089e-01 -2.49869645e-01  6.42519057e-01
  1.60114065e-01 -4.03126806e-01 -4.21593785e-01 -2.51827121e-01
  2.24453896e-01 -3.69

In [51]:
# f = open('searcher/static/searcher/bert_method_data/Reddit_umap_kmeans.json', 'w')
# eg.doc_embs['emb'] = []
# json.dump(eg.doc_embs, f)
# f.close()